# Xavier Deep network version

# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.6.0


# checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model (Class Version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function Version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model, optimizer, writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.6428, train_loss: 2.02296805, train_accuracy: 0.5469, test_Accuracy: 0.3043
Epoch: [ 0] [    1/  468] time: 0.9049, train_loss: 1.99152493, train_accuracy: 0.5234, test_Accuracy: 0.4825
Epoch: [ 0] [    2/  468] time: 1.2032, train_loss: 1.81742263, train_accuracy: 0.7891, test_Accuracy: 0.6640
Epoch: [ 0] [    3/  468] time: 1.5108, train_loss: 1.57618630, train_accuracy: 0.7109, test_Accuracy: 0.6823
Epoch: [ 0] [    4/  468] time: 1.7835, train_loss: 1.26749468, train_accuracy: 0.7812, test_Accuracy: 0.7337
Epoch: [ 0] [    5/  468] time: 2.0603, train_loss: 1.10239744, train_accuracy: 0.7109, test_Accuracy: 0.7264
Epoch: [ 0] [    6/  468] time: 2.3747, train_loss: 0.81278586, train_accuracy: 0.8203, test_Accuracy: 0.7273
Epoch: [ 0] [    7/  468] time: 2.6803, train_loss: 0.76642001, train_accuracy: 0.7500, test_Accuracy: 0.7475
Epoch: [ 0] [    8/  468] time: 2.9951,

Epoch: [ 0] [   74/  468] time: 25.1943, train_loss: 0.36911723, train_accuracy: 0.9062, test_Accuracy: 0.9267
Epoch: [ 0] [   75/  468] time: 25.4942, train_loss: 0.25182772, train_accuracy: 0.9375, test_Accuracy: 0.9220
Epoch: [ 0] [   76/  468] time: 25.8183, train_loss: 0.24461502, train_accuracy: 0.9219, test_Accuracy: 0.9170
Epoch: [ 0] [   77/  468] time: 26.1520, train_loss: 0.23901287, train_accuracy: 0.9297, test_Accuracy: 0.9205
Epoch: [ 0] [   78/  468] time: 26.4669, train_loss: 0.14750201, train_accuracy: 0.9609, test_Accuracy: 0.9296
Epoch: [ 0] [   79/  468] time: 26.7531, train_loss: 0.17702627, train_accuracy: 0.9297, test_Accuracy: 0.9321
Epoch: [ 0] [   80/  468] time: 27.0402, train_loss: 0.08914700, train_accuracy: 0.9766, test_Accuracy: 0.9335
Epoch: [ 0] [   81/  468] time: 27.3197, train_loss: 0.11015289, train_accuracy: 0.9688, test_Accuracy: 0.9334
Epoch: [ 0] [   82/  468] time: 27.5999, train_loss: 0.12408970, train_accuracy: 0.9609, test_Accuracy: 0.9306
E

Epoch: [ 0] [  148/  468] time: 46.7144, train_loss: 0.10684206, train_accuracy: 0.9766, test_Accuracy: 0.9484
Epoch: [ 0] [  149/  468] time: 46.9944, train_loss: 0.20566784, train_accuracy: 0.9453, test_Accuracy: 0.9451
Epoch: [ 0] [  150/  468] time: 47.2640, train_loss: 0.22879004, train_accuracy: 0.9375, test_Accuracy: 0.9457
Epoch: [ 0] [  151/  468] time: 47.5410, train_loss: 0.10958506, train_accuracy: 0.9688, test_Accuracy: 0.9477
Epoch: [ 0] [  152/  468] time: 47.8683, train_loss: 0.09823273, train_accuracy: 0.9766, test_Accuracy: 0.9483
Epoch: [ 0] [  153/  468] time: 48.1486, train_loss: 0.12221617, train_accuracy: 0.9766, test_Accuracy: 0.9482
Epoch: [ 0] [  154/  468] time: 48.5126, train_loss: 0.16499691, train_accuracy: 0.9531, test_Accuracy: 0.9516
Epoch: [ 0] [  155/  468] time: 48.8186, train_loss: 0.11586575, train_accuracy: 0.9766, test_Accuracy: 0.9534
Epoch: [ 0] [  156/  468] time: 49.1184, train_loss: 0.10873608, train_accuracy: 0.9688, test_Accuracy: 0.9524
E

Epoch: [ 0] [  222/  468] time: 68.3566, train_loss: 0.14081053, train_accuracy: 0.9453, test_Accuracy: 0.9525
Epoch: [ 0] [  223/  468] time: 68.6647, train_loss: 0.10570596, train_accuracy: 0.9688, test_Accuracy: 0.9508
Epoch: [ 0] [  224/  468] time: 68.9596, train_loss: 0.14268619, train_accuracy: 0.9609, test_Accuracy: 0.9485
Epoch: [ 0] [  225/  468] time: 69.2537, train_loss: 0.11961171, train_accuracy: 0.9688, test_Accuracy: 0.9458
Epoch: [ 0] [  226/  468] time: 69.5411, train_loss: 0.11108111, train_accuracy: 0.9766, test_Accuracy: 0.9437
Epoch: [ 0] [  227/  468] time: 69.8318, train_loss: 0.07335425, train_accuracy: 0.9844, test_Accuracy: 0.9411
Epoch: [ 0] [  228/  468] time: 70.1366, train_loss: 0.13840327, train_accuracy: 0.9766, test_Accuracy: 0.9409
Epoch: [ 0] [  229/  468] time: 70.4417, train_loss: 0.09548021, train_accuracy: 0.9766, test_Accuracy: 0.9455
Epoch: [ 0] [  230/  468] time: 70.7308, train_loss: 0.10873476, train_accuracy: 0.9766, test_Accuracy: 0.9544
E

Epoch: [ 0] [  296/  468] time: 90.8263, train_loss: 0.15474902, train_accuracy: 0.9531, test_Accuracy: 0.9554
Epoch: [ 0] [  297/  468] time: 91.1045, train_loss: 0.11675431, train_accuracy: 0.9688, test_Accuracy: 0.9528
Epoch: [ 0] [  298/  468] time: 91.3821, train_loss: 0.07109351, train_accuracy: 0.9922, test_Accuracy: 0.9547
Epoch: [ 0] [  299/  468] time: 91.6599, train_loss: 0.06932019, train_accuracy: 0.9688, test_Accuracy: 0.9548
Epoch: [ 0] [  300/  468] time: 91.9401, train_loss: 0.13686465, train_accuracy: 0.9688, test_Accuracy: 0.9554
Epoch: [ 0] [  301/  468] time: 92.2152, train_loss: 0.09186224, train_accuracy: 0.9609, test_Accuracy: 0.9538
Epoch: [ 0] [  302/  468] time: 92.4962, train_loss: 0.06017989, train_accuracy: 0.9922, test_Accuracy: 0.9539
Epoch: [ 0] [  303/  468] time: 92.8537, train_loss: 0.12728861, train_accuracy: 0.9375, test_Accuracy: 0.9560
Epoch: [ 0] [  304/  468] time: 93.1748, train_loss: 0.17717788, train_accuracy: 0.9453, test_Accuracy: 0.9556
E

Epoch: [ 0] [  370/  468] time: 111.4623, train_loss: 0.08176406, train_accuracy: 0.9766, test_Accuracy: 0.9653
Epoch: [ 0] [  371/  468] time: 111.7386, train_loss: 0.08880336, train_accuracy: 0.9609, test_Accuracy: 0.9665
Epoch: [ 0] [  372/  468] time: 112.0230, train_loss: 0.09172840, train_accuracy: 0.9688, test_Accuracy: 0.9653
Epoch: [ 0] [  373/  468] time: 112.2807, train_loss: 0.09537917, train_accuracy: 0.9766, test_Accuracy: 0.9654
Epoch: [ 0] [  374/  468] time: 112.5719, train_loss: 0.08634134, train_accuracy: 0.9844, test_Accuracy: 0.9634
Epoch: [ 0] [  375/  468] time: 112.8433, train_loss: 0.07856734, train_accuracy: 0.9688, test_Accuracy: 0.9650
Epoch: [ 0] [  376/  468] time: 113.1270, train_loss: 0.08114114, train_accuracy: 0.9766, test_Accuracy: 0.9656
Epoch: [ 0] [  377/  468] time: 113.3997, train_loss: 0.07466333, train_accuracy: 0.9766, test_Accuracy: 0.9676
Epoch: [ 0] [  378/  468] time: 113.6780, train_loss: 0.07567351, train_accuracy: 0.9922, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 132.6703, train_loss: 0.09066272, train_accuracy: 0.9609, test_Accuracy: 0.9600
Epoch: [ 0] [  445/  468] time: 132.9632, train_loss: 0.12509516, train_accuracy: 0.9609, test_Accuracy: 0.9579
Epoch: [ 0] [  446/  468] time: 133.2686, train_loss: 0.10177431, train_accuracy: 0.9688, test_Accuracy: 0.9589
Epoch: [ 0] [  447/  468] time: 133.5595, train_loss: 0.14110211, train_accuracy: 0.9375, test_Accuracy: 0.9644
Epoch: [ 0] [  448/  468] time: 133.8317, train_loss: 0.07927982, train_accuracy: 0.9766, test_Accuracy: 0.9642
Epoch: [ 0] [  449/  468] time: 134.1028, train_loss: 0.08254196, train_accuracy: 0.9766, test_Accuracy: 0.9623
Epoch: [ 0] [  450/  468] time: 134.3831, train_loss: 0.05300292, train_accuracy: 0.9766, test_Accuracy: 0.9582
Epoch: [ 0] [  451/  468] time: 134.6620, train_loss: 0.10731838, train_accuracy: 0.9688, test_Accuracy: 0.9571
Epoch: [ 0] [  452/  468] time: 134.9551, train_loss: 0.16017225, train_accuracy: 0.9609, test_Accuracy: